# Packages Installation

In [ ]:
import pandas as pd
import xarray as xr
import datetime as dt

# Load local modules
!pip install C:/Users/jessy/Documents/repositories/hakai-profile-qaqc
import hakai_qc


# Download station to be reviewed

In [33]:
# Get Hakai CTD Data Download through the API
station = 'QU39'
start_time = '2019-01-01'
end_time = '2020-01-01'

variable_lists = hakai_qc.get.hakai_api_selected_variables()

# Let's just get the data from QU39
filterUrl = 'station='+station+'&status!=MISCAST&limit=-1'+\
    '&measurement_dt>'+start_time+'&measurement_dt<'+end_time+\
    '&fields='+','.join(variable_lists)
df, url = hakai_qc.get.hakai_ctd_data(filterUrl)
print(str(len(df))+' records found')

# Regroup profiles and cast direction and sort them by pressure
group_variables = ['device_model','device_sn','ctd_file_pk','ctd_cast_pk','direction_flag']
df = df.sort_values(by=group_variables+['pressure'])

# Just show the first few lines to have a look
df.head() # Show the top of the data frame

16162 records found


,ctd_file_pk,ctd_cast_pk,hakai_id,ctd_data_pk,filename,device_model,device_sn,work_area,cruise,station,...,sos_un,sos_un_flag,backscatter_beta,backscatter_beta_flag,cdom_ppb,cdom_ppb_flag,absolute salinity,conservative temperature,density,sigma0
5237,5505,13723,080217_2019-04-02T19:41:11.666Z,8614263,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,...,None,None,None,None,None,None,29.086675,10.196213,1022.229070,22.219975
5238,5505,13723,080217_2019-04-02T19:41:11.666Z,8614264,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,...,None,None,None,None,None,None,29.112250,10.055276,1022.275465,22.261813
5239,5505,13723,080217_2019-04-02T19:41:11.666Z,8614265,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,...,None,None,None,None,None,None,29.218211,9.728579,1022.412415,22.394186
5240,5505,13723,080217_2019-04-02T19:41:11.666Z,8614266,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,...,None,None,None,None,None,None,29.336029,9.254311,1022.579585,22.556750
5241,5505,13723,080217_2019-04-02T19:41:11.666Z,8614267,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,...,None,None,None,None,None,None,29.363512,9.053406,1022.635010,22.607581


# Review Profile

In [6]:
# Get the list of parameters (variables associated with a *_flag column), ignore direction_flag
flag_columns = df.filter(like='_flag').columns.tolist()
var_columns = set([sub.replace('_flag','') for sub in flag_columns])-{'direction'}

# Temporary until the database has the right setup:
for var in var_columns:
    if var+'_qartod_flag' not in df.columns:
        df[var+'_qartod_flag'] = 9
    if var+'_flag_description' not in df.columns:
        df[var+'_flag_description'] = ''
        
# Retrive list of tests
test_list = sorted(df.filter(regex='_flag$|_test$').columns.tolist())
test_list.append("None")

# Run the interactive profile review tool
hakai_qc.show.interactive_profile_viewer(df,var_columns,test_list)

Select Flagged Variable(s) to consider in the following list:


interactive(children=(Dropdown(description='Test to review', index=73, options=('backscatter_beta_description_…

In [ ]:
# Generate a NetCDF File for the approuved profiles!

In [30]:
approuved_hakai_id = ['080217_2017-01-08T18:03:05.167Z','080217_2017-01-26T16:56:39.000Z']

save_to = ''
creator = {
    'creator_name': 'Jessy Barrette',
    'creator_url': 'https://hakai.org/',
    'creator_email': 'jessy.barrette@hakai.org',
    'creator_institution':'Hakai Institute',
    'creator_country':'Canada'
}
publisher = {
    'publisher_name': 'Hakai Institute',
    'publisher_url': 'https://hakai.org/',
    'publisher_email': 'contact@hakai.org',
    'publisher_country':'Canada',
    'publisher_institution':'Hakai Institute'
}

global_attributes = {'institution':'Hakai Institute',
                     'project':'Hakai Oceanography',
                     'title':'Hakai Research CTD Profile: '+hakai_id,
                     'summary':'text describing that specific data',
                     'comment': '',
                     'infoUrl':'https://hakai.org/',
                     'keywords':'conductivity,temperature,salinity,depth,pressure,dissolved oxygen',
                     'acknowledgment':'Hakai Field Techniciens, research and IT groups',
                     'id':hakai_id,
                     'naming_authority':'Hakai Instititute',
                     'date_created':str(dt.datetime.utcnow().isoformat()),
                     'standard_name_vocabulary': 'CF Standard Name Table v1.3',
                     'license':'unknown',
                     'geospatial_lat_units':'degrees_north',
                     'geospatial_lon_units':'degrees_east'}
                     
extra_global_attributes = global_attributes
extra_global_attributes.update(creator)
extra_global_attributes.update(publisher)

# Create the NetCDF Source file!
hakai_qc.get.research_profile_netcdf(hakai_id,save_to,
                                     extra_global_attributes=extra_global_attributes,
                                     extra_variable_attributes=variable_attributes)